# RGI01 outlines selection

In [68]:
import geopandas as gpd
from oggm import utils
import subprocess

## Read the file 

In [ ]:
idir = '../l2_sel_reg_tars/'

In [32]:
# Read L2 files
shp = gpd.read_file('tar://' + idir + 'RGI01.tar.gz/RGI01/RGI01.shp')

## Outline selection 

In [33]:
# List analysts
shp.analysts.unique()

array(['Beedle, Matthew', 'Manley, William', 'Herreid, Samuel',
       'Frey, Holger; Le Bris, Raymond; Paul, Frank', 'Rich, Justin',
       'Bolch, Tobias',
       'Beedle, Matthew; Berthier, Etienne; Bolch, Tobias; Burgess, Evan; Cogley, Graham; Forster, Richard; Giffen, Bruce A.; Hall, Dorothy K.; Kienholz, Christian; LeBris, Raymond; Manley, William',
       'Maraldo, Dean'], dtype=object)

For this region selecting glaciers can be done either by analyst or by "proc_desc":

In [36]:
sel_analysts = ('Beedle, Matthew; Berthier, Etienne; Bolch, Tobias; Burgess, Evan; Cogley, Graham; '
                'Forster, Richard; Giffen, Bruce A.; Hall, Dorothy K.; Kienholz, Christian; LeBris, Raymond; Manley, William')
len(shp.loc[shp.analysts == sel_analysts])

27106

In [37]:
sel_proc_desc = ('Semi-automated glacier classification.; From Randolph Glacier Inventory Version 5.0. Source file: 01_rgi50_Alaska.zip.  '
                 'See http://www.glims.org/RGI/.  GLIMS IDs were adjusted and metadata was added at NSIDC.')
len(shp.loc[shp.proc_desc == sel_proc_desc])

27106

In [57]:
rgi7 = shp.loc[shp.analysts == sel_analysts]

## Consistency check with RGI6 

In [54]:
import zipfile
with zipfile.ZipFile('../l0_RGIv6/01_rgi60_Alaska.zip', "r") as z:
    for f in z.filelist:
        print(f.filename)

01_rgi60_Alaska.dbf
01_rgi60_Alaska.prj
01_rgi60_Alaska.shp
01_rgi60_Alaska.shx
01_rgi60_Alaska_hypso.csv


In [56]:
rgi6 = gpd.read_file('zip://../l0_RGIv6/01_rgi60_Alaska.zip/01_rgi60_Alaska.shp')

In [58]:
len(rgi7), len(rgi6)

(27106, 27108)

There are two missing glaciers! One has been wrongly attributed to Region 1 in the RGI6 (`RGI60-01.05285`), another we don't know yet.

In [71]:
rgi6_cor = rgi6.loc[rgi6.RGIId != 'RGI60-01.05285']

In [72]:
rgi7['area'].sum() * 1e-6, rgi6['Area'].sum()

(86724.5510826829, 86725.053)

Not bad! So the remaining missing glacier must be very small...

## Write out and tar 

In [66]:
reg = 1
odir = utils.mkdir('../l3_rgi7b')
odir_tar = utils.mkdir('../l3_rgi7b_tar')

In [69]:
dd = utils.mkdir(f'{odir}/RGI{reg:02d}/', reset=True)

print('Writing...')
rgi7.to_file(dd + f'RGI{reg:02d}.shp')

print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{odir_tar}/RGI{reg:02d}.tar.gz', '-C', odir, f'RGI{reg:02d}']))

Writing...
Taring...
CompletedProcess(args=['tar', '-zcvf', '../l3_rgi7b_tar/RGI01.tar.gz', '-C', '../l3_rgi7b', 'RGI01'], returncode=0)
